In [1]:
import torch
import sys
sys.path.append('../')
sys.path.append('../../')
from lstm_agent_cql_bc import DecisionLSTM
import yaml
import argparse
from tqdm import tqdm
import numpy as np
from tqdm import tqdm
import os

import matplotlib.pyplot as plt

import torch
import numpy as np

import pickle
from tqdm import tqdm
#import env_vizdoom2
import matplotlib.pyplot as plt
from itertools import count
import time
import random
from scipy.stats import sem

import torch
import numpy as np
import gym
gym.logger.set_level(40)
import logging
logging.disable(logging.WARNING)
from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning, message='`np.bool` is a deprecated alias')
import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_model(seed, exp_name, loss_mode, stacked_input):
    agent = DecisionLSTM(3, 6, 128, mode='memory_maze')
    
    run_name = f'{exp_name}_{loss_mode}_{seed}_stacked_{stacked_input}'
    print(run_name)
    model_path = f'../ckpt/memory_maze_ckpt/{loss_mode}/{seed}/{run_name}.ckpt'
    
    agent.load_state_dict(torch.load(model_path))
    
    agent.eval()
    agent.to(agent.device)
    
    return agent

In [3]:
exp_name = 'memory_maze'
stacked_input = False
loss_mode = 'bc'

In [ ]:
totals = []

for i in range(3, 3+1):
    agent = load_model(
        seed=i,
        exp_name=exp_name,
        stacked_input=stacked_input,
        loss_mode=loss_mode, 
    )

    _ = agent.eval()
    _ = agent.to(agent.device)
    PATH = f'../memory_maze_ckpt/{loss_mode}/{i}/{exp_name}_{loss_mode}_{i}_stacked_{stacked_input}.ckpt'

    device = 'cuda:0'
    EPISODE_TIMEOUT = 1000 # 90

    NUMBER_OF_TRAIN_DATA = 100
    returns_red, returns_green, returns_total = [], [], []
    agent.eval()
    

    for j in tqdm(range(NUMBER_OF_TRAIN_DATA)):
        obsList, actList, rewList, doneList, isRedList = [], [], [], [], []
        env = gym.make('memory_maze:MemoryMaze-9x9-v0', seed=j)
        times = []
        state = env.reset()
        # plt.imshow(obs['image'].transpose(1,2,0))
        # plt.show()
        state = state.transpose(2, 0, 1) # model trained on [C, H, W], but env returns [H, W, C]
        # state = state.reshape(1, 1, state.shape[0], state.shape[1], state.shape[2])

        mask = torch.ones(1,1).to(device)
        done = False
        agent.init_hidden(1)
        action = 0
        rtg = 18.1

        for t in count():
            times.append(t)
            #result = policy(torch.from_numpy(obs['image']).unsqueeze(0).to(device), state, mask)
            #action, state = result['actions'], result['states']

            states = torch.from_numpy(state).unsqueeze(0).unsqueeze(0).to(device).float()

            with torch.no_grad():
                q_values = []
                for possible_action in range(0, 6): 
                    action_tensor = torch.tensor([[[possible_action]]], 
                                            dtype=torch.float32, 
                                            device=device).long()
                    rtg_tensor = torch.tensor([[[rtg]]], 
                                            dtype=torch.float32, 
                                            device=device)#.long()
                    if loss_mode == 'cql':
                        update_lstm_hidden = possible_action==5
                    else:
                        update_lstm_hidden = True
                    
                    action_preds, q1, q2, _ = agent.forward(
                        states = states.float() / 255.,
                        actions = action_tensor,
                        returns_to_go = rtg_tensor,
                        update_hidden = update_lstm_hidden,
                        stacked_input = stacked_input,
                    )
                    q_value = torch.minimum(q1, q2)
                    q_values.append(q_value)

                    if not loss_mode == 'cql':
                        break

                # Select action with max Q-value
                if loss_mode == 'cql':
                    q_values = torch.cat(q_values, dim=-1)
                    action = torch.argmax(q_values).item() #+ 3
                else:
                    action = torch.argmax(torch.softmax(action_preds, dim=-1).squeeze()).item()

            #action = random.choice([3,4])
            #print(t,action, q_values)
            # print(action)
            state, reward, done, info = env.step(action)
            # plt.imshow(state)
            # plt.show()
            state = state.transpose(2, 0, 1)
            rtg -= reward

            rewList.append(reward)
            actList.append(action)
            doneList.append(int(done))
            

            if done or t == EPISODE_TIMEOUT-1:
                returns_total.append(np.sum(rewList))
                print(np.sum(rewList))
                break


    print(f"\nResults for checkpoint {i}:")
    print(f"Total average return:      {np.mean(returns_total):.2f}")
    print("-" * 50)

    totals.append(np.mean(returns_total))


print('\n')
print('#'*50)

print(f'TOTAL: {np.mean(totals)} ± {sem(totals)}')
# (18+0+0+3+13+11+2+9+3+13+5+1+6+3)/14

memory_maze_bc_3_stacked_False


  1%|          | 1/100 [00:44<1:13:54, 44.80s/it]

19.0


  2%|▏         | 2/100 [01:30<1:13:57, 45.28s/it]

0.0


...

12.0


 16%|█▌        | 16/100 [11:50<1:01:58, 44.27s/it]

5.0


 17%|█▋        | 17/100 [12:34<1:01:07, 44.18s/it]

8.0
